In [1]:
import numpy as np
import pandas as pd
import pickle


movies = pd.read_csv('./dataset/tmdb_5000_movies.csv')
credits = pd.read_csv('./dataset/tmdb_5000_credits.csv')

# movies.head()

In [2]:
movies.merge(credits, on = 'title').shape

(4809, 23)

In [3]:
movies = movies[['id', 'title', 'overview', 'genres', 'keywords']]

movies.isnull().sum()

movies.dropna(inplace=True)

movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [4]:
import ast
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            return L

movies['genres'] = movies['genres'].apply(convert)

movies.head()

,id,title,overview,genres,keywords
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."


In [5]:
new_df = movies[['id', 'title', 'genres', 'overview']]

### Vectorize the inputs here 

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer (max_features=5000, stop_words='english')


In [7]:
vectors = cv.fit_transform(new_df['overview']).toarray()
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zombies', 'zone', 'zoo'], dtype=object)

In [8]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vectors)

similarity[0].shape

(4800,)

## Main function

In [10]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x : x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [11]:
recommend('Batman Begins')

Batman Forever
Raising Helen
Synecdoche, New York
Batman
The Dark Knight Rises


In [12]:

pickle.dump(new_df.to_dict(), open('./dataset/movie_dict.pkl','wb'))
pickle.dump(similarity,open('./dataset/movie_similarity.pkl','wb'))